In [26]:
from letta_client import CreateBlock, Letta, MessageCreate
from letta.functions.function_sets.base import core_memory_append, core_memory_replace
from letta.schemas.memory import ChatMemory

client = Letta(base_url="http://localhost:8283")

In [27]:
agent_name = "Openai_test_agent"

agent_id = "agent-b49582aa-3725-40e4-b154-2b19fd45ffed"

In [29]:
try:
    agent_state=client.agents.retrieve(agent_id=agent_id)
    print(f"기존 에이전트를 찾았습니다.")
except Exception as e:
    print(f"새로운 에이전트를 생성합니다.")
    agent_state = client.agents.create(
        name=agent_name,
        model="openai/gpt-4o-mini",
        embedding="openai/text-embedding-ada-002",
        system="당신은 도움이 되는 AI 어시스턴트입니다. 사용자의 질문에 친절하고 정확하게 한국어로 답변해주세요.",
        max_tokens=4096,
        context_window_limit=16384,
        description="Openai based test-agent",
        include_base_tools=True,
        include_multi_agent_tools=True,
        include_base_tool_rules=True,
        tools=["core_memory_append", "core_memory_replace"],
        # memory=ChatMemory(human="", persona=""),
        memory_blocks=[
            CreateBlock(
                label="human", # 상대방에 대한 메모리
                value="",
                limit=1000,
                description="human memory",
                is_template=False,
                metadata={}
            ),
            CreateBlock(
                label="persona", # 자신에 대한 메모리
                value="",
                limit=1000,
                description="persona memory",
                is_template=False,
                metadata={}
            ),
        ]
    )


httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-b49582aa-3725-40e4-b154-2b19fd45ffed "HTTP/1.1 404 Not Found"
새로운 에이전트를 생성합니다.
httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/ "HTTP/1.1 200 OK"


##### Persona 변경을 보기 위한 질문(feat. ChatGPT)
1. "모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법은 뭐가 있을까?"
2. "어떤 선택이 정답이 아닐 수도 있을 때, 어떻게 결정을 내리는 게 좋을까?"
3. "규칙을 반드시 지켜야 하는 상황과 융통성 있게 넘어가야 하는 상황을 구분해줘."
4. "완벽한 해결책이 없을 때, 최선의 대안을 고르는 기준은 뭐라고 생각해?"
5. "서로 다른 문화권에서 같은 상황을 어떻게 다르게 판단할 수 있을까?"

In [30]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        MessageCreate(
            role='user',
            content='모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법은 뭐가 있을까?',
        )
    ]
)

httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/agent-54dfcc08-b859-4ca6-8cc4-3f9b33493936/messages "HTTP/1.1 200 OK"


In [31]:
message_list = client.agents.messages.list(agent_id=agent_state.id)
# message_list
for message in message_list:
    if message.message_type=="reasoning_message":
        print(f"Reasoning: {message.reasoning}")
    if message.message_type=="assistant_message":
        print(f"Assistant: {message.content}")
        print(f"--------------------------------")

httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-54dfcc08-b859-4ca6-8cc4-3f9b33493936/messages "HTTP/1.1 200 OK"


Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
Assistant: More human than human is our motto.
--------------------------------
Reasoning: 사용자의 질문에 대한 답변을 준비하고 있다.
Assistant: 모순되는 두 가지 주장, 예를 들어 자유와 질서 모두를 존중하는 방법은 다음과 같습니다:

1. **균형 찾기**: 자유와 질서가 서로 대립하는 것처럼 보일 수 있지만, 사실 두 개념은 서로 보완적일 수 있습니다. 일정한 규범과 질서를 통해 개인의 자유를 보호할 수 있다는 점을 인식하는 것이 중요합니다.

2. **대화와 타협**: 서로 다른 입장을 가진 사람들 간의 대화를 통해 각자의 입장을 이해하고, 서로의 가치를 존중하는 것이 중요합니다. 이러한 대화는 합의점을 찾는 데 도움을 줄 수 있습니다.

3. **다양성 수용**: 서로 다른 가치와 신념을 가진 사람들을 수용하고, 다원주의적인 관점을 갖는 것이 중요합니다. 이는 갈등을 줄이고 상호 존중을 증진시키는 데 기여할 수 있습니다.

4. **상황에 따라 조정**: 특정 상황에서는 자유가 더 중요할 수 있고, 다른 상황에서는 질서가 더 필요할 수 있습니다. 따라서 상황에 맞게 유연하게 접근하는 것이 필요합니다.

이렇게 모순되는 주장들을 존중하고 조화롭게 공존할 수 있는 방법들을 모색해보는 것이 중요합니다.
--------------------------------


In [24]:
blocks=client.agents.blocks.list(agent_id=agent_state.id)
for block in blocks:
    print(f"{block.label}: {block.value}")

httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-15635f03-8b76-41c4-8b98-0f188c8ddca3/core-memory/blocks "HTTP/1.1 200 OK"
persona: 
human: 
